### Applied Data Science Capstone Week 3 - 3

In [1]:
from bs4 import BeautifulSoup
import pandas as pd

import numpy as np # library to handle data in a vectorized manner


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


Download data.

In [2]:
!wget -q -O 'List_of_postal_codes_of_Canada.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
# postalcode - geographical coordinates
!wget -q -O 'geopostal.csv' https://cocl.us/Geospatial_data
    
!wget -q -O 'tront.geojson' https://geo.nyu.edu/download/file/nyu-2451-36180-geojson.json

Read a downloaded geojson.

In [3]:
with open('tront.geojson') as json_data:
    tront_geojson = json.load(json_data)
#newyork_data

Read rows in file that was downloaded from http://cocl.us/Geospatial_data except for the top row.

In [4]:
import csv
postal_to_geographica_coordinates = {}
with open('geopostal.csv') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    
    for index, row in enumerate(csvreader):
        if index > 0 :  postal_to_geographica_coordinates.setdefault(row[0], (row[1],row[2]))
#postal_to_geographica_coordinates            

Function tp get text in table's cell is in &lt;td&gt;abc&lt;/td&gt; or &lt;td&gt;&lt;a&gt;abc&lt;/a&gt;&lt;/td&gt;.

In [5]:
def get_text_in_tag(tag):
    """
    get text in like <td>aaa</td> or <td><a>aaa</a></td> from argument tag.
    """
    if tag.string is not None:
        return tag.string
    else:
        return tag.find('a').string

                             

Load a downloaded wiki page.

In [6]:
with open('List_of_postal_codes_of_Canada.html') as html_file:
    soup = BeautifulSoup(html_file,"lxml")

Read the 2nd or later rows in table and create dictionary.<br/>
When postalcode as key is in dictionary alaready, add Neighbourhood to array in value of dictionary.

In [7]:
not_assigned = 'Not assigned'
table_header=[ 'Neighborhood', 'PostalCode', 'Borough', 'Latitude', 'Longitude', 'IsTront']

table_values = []
postcode_dict = {} 

table_tag = soup.table
first_row = table_tag.find('tr')

for tr in first_row.find_next_siblings():
    row = [ get_text_in_tag(td) if td.string is None else td.string.strip() for td in tr.find_all('td')]

    if row[1] == not_assigned:
        continue
    elif row[2] == not_assigned:
        postcode_dict.setdefault(row[1],  [row[0], row[1], postal_to_geographica_coordinates[row[0]], row[1].find('Tront') > -1  ])
    else:
        postcode_dict.setdefault(row[2],  [row[0], row[1], postal_to_geographica_coordinates[row[0]], row[1].find('Toronto') > -1  ])
        
data_src = [ [key, value[0], value[1], float(value[2][0]), float(value[2][1]), value[3] ] for key,value in postcode_dict.items()]     

Create a DataFrame.

In [8]:
neighborhoods = pd.DataFrame(data_src, columns=table_header)
print(neighborhoods.shape)
neighborhoods.head()

(210, 6)


,Neighborhood,PostalCode,Borough,Latitude,Longitude,IsTront
0,Eringate,M9C,Etobicoke,43.643515,-79.577201,False
1,Harbord,M5S,Downtown Toronto,43.662696,-79.400049,True
2,Trinity,M6J,West Toronto,43.647927,-79.419750,True
3,Cliffside,M1M,Scarborough,43.716316,-79.239476,False
4,Morningside,M1E,Scarborough,43.763573,-79.188711,False


Get the geograpical coordinate of Tronto city. 

In [9]:
address = 'Toronto city'

geolocator = Nominatim()
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(toronto_latitude, toronto_longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [10]:
toronto_data = neighborhoods.loc[neighborhoods['IsTront'] == True]
toronto_data.head()

,Neighborhood,PostalCode,Borough,Latitude,Longitude,IsTront
1,Harbord,M5S,Downtown Toronto,43.662696,-79.400049,True
2,Trinity,M6J,West Toronto,43.647927,-79.419750,True
7,Harbourfront East,M5J,Downtown Toronto,43.640816,-79.381752,True
10,Ryerson,M5B,Downtown Toronto,43.657162,-79.378937,True
13,Stn A PO Boxes 25 The Esplanade,M5W,Downtown Toronto,43.646435,-79.374846,True


Show neighborhoods of Toronto city

In [11]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = toronto_data['PostalCode']
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [12]:
# @hidden_cell
CLIENT_ID = 'SCXVI3PF4XNWFUT0FM54TC0GTSS1JHEQ3PT500M421AGHWH5' # your Foursquare ID
CLIENT_SECRET = 'RVTD4SO1HUNBLGNWN04KSU1WKRTDXMB41ELHKT5V4YDKMPEO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT=100

The function to repeat the same process to all the neighborhood

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get venues in Tronto via foursquare API.

In [14]:
toronto_venues = getNearbyVenues( names = toronto_data['Neighborhood'],
                                   latitudes = toronto_data['Latitude'],
                                   longitudes = toronto_data['Longitude']
                                  )

Harbord
Trinity
Harbourfront East
Ryerson
Stn A PO Boxes 25 The Esplanade
First Canadian Place
Davisville
The Annex
CN Tower
Garden District
Commerce Court
Swansea
Davisville North
Richmond
South Hill
Dufferin
Dovercourt Village
Roncesvalles
Forest Hill North
Parkdale Village
Rathnelly
Toronto Islands
Central Bay Street
Little Portugal
Lawrence Park
Roselawn
Church and Wellesley
The Beaches
Bathurst Quay
Underground city
Kensington Market
South Niagara
The Danforth West
Forest Hill SE
Forest Hill West
Design Exchange
Cabbagetown
University of Toronto
Riverdale
Business reply mail Processing Centre969 Eastern
Summerhill West
Brockton
Parkdale
Railway Lands
Harbourfront
Exhibition Place
King
King and Spadina
North Midtown
The Beaches West
Victoria Hotel
Chinatown
St. James Town
Toronto Dominion Centre
Island airport
Christie
Union Station
High Park
Berczy Park
Deer Park
The Junction South
Yorkville
Regent Park
Rosedale
North Toronto West
India Bazaar
Summerhill East
Grange Park
Harbourfr

In [15]:
print(toronto_venues.shape)
toronto_venues.head()

(3205, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbord,43.662696,-79.400049,Yasu,43.662837,-79.403217,Japanese Restaurant
1,Harbord,43.662696,-79.400049,Rasa,43.662757,-79.403988,Restaurant
2,Harbord,43.662696,-79.400049,Piano Piano,43.662949,-79.402898,Italian Restaurant
3,Harbord,43.662696,-79.400049,Cafe Cancan,43.662735,-79.403447,French Restaurant
4,Harbord,43.662696,-79.400049,Almond Butterfly,43.662836,-79.403365,Bakery


Check how many venues were returned for each neighborhood

In [16]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,13,13,13,13,13,13
Berczy Park,55,55,55,55,55,55
Brockton,21,21,21,21,21,21
Business reply mail Processing Centre969 Eastern,19,19,19,19,19,19
CN Tower,13,13,13,13,13,13
Cabbagetown,45,45,45,45,45,45
Central Bay Street,81,81,81,81,81,81
Chinatown,98,98,98,98,98,98


Let's find out how many unique categories can be curated from all the returned venues

In [17]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


Analyze Each Neighborhood

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbord,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

examine the new dataframe size.

In [19]:
toronto_onehot.shape

(3205, 237)

group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.000000,0.010000,0.010000,0.000000,0.030000,0.0000,0.000000,0.000000,0.0000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.0,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.010000,0.060000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000

show shape

In [21]:
toronto_grouped.shape

(72, 237)

print each neighborhood along with the top 5 most common venues

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.15
1   Airport Service  0.15
2  Airport Terminal  0.15
3     Boat or Ferry  0.08
4          Boutique  0.08


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1          Restaurant  0.05
2        Cocktail Bar  0.05
3          Steakhouse  0.04
4  Italian Restaurant  0.04


----Brockton----
            venue  freq
0  Breakfast Spot  0.10
1            Café  0.10
2     Coffee Shop  0.10
3             Gym  0.05
4   Burrito Place  0.05


----Business reply mail Processing Centre969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2          Restaurant  0.05
3          Smoke Shop  0.05
4                 Spa  0.05


----CN Tower----
              venue  freq
0   

Write a function to sort the venues in descending order.

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

reate the new dataframe and display the top 10 venues for each neighborhood.

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Gym,Cosmetics Shop,Restaurant,Hotel,Bar
1,Bathurst Quay,Airport Terminal,Airport Service,Airport Lounge,Boutique,Airport Gate,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport,Harbor / Marina
2,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Café,Pub,Steakhouse,Farmers Market,Bakery,Italian Restaurant
3,Brockton,Breakfast Spot,Café,Coffee Shop,Yoga Studio,Pet Store,Burrito Place,Caribbean Restaurant,Climbing Gym,Performing Arts Venue,Bar
4,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Moving Target,Recording Studio,Restaurant,Burrito Place,Brewery
5,CN Tower,Airport Terminal,Airport Service,Airport Lounge,Boutique,Airport Gate,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport,Harbor / Marina
6,Cabbagetown,Restaurant,Coffee Shop,Pub,Café,Bakery,Pizza Place,Italian Restaurant,Indian Restaurant,Bistro,Butcher
7,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Chinese Restaurant,Bar,Ice Cream Shop,Burger Joint,Sandwich Place,Thai Restaurant,Bubble Tea Shop
8,Chinatown,Café,Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Coffee Shop,Bakery,Chinese Restaurant,Mexican Restaurant,Comfort Food Restaurant,Gaming Cafe
9,Christie,Café,Grocery Store,Park,Italian Restaurant,Athletics & Sports,Restaurant,Diner,Nightclub,Baby Store,Coffee Shop


# Crustering

Run k-means to cluster the neighborhood into 5 clusters.

In [25]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [26]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,PostalCode,Borough,Latitude,Longitude,IsTront,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Harbord,M5S,Downtown Toronto,43.662696,-79.400049,True,2,Café,Coffee Shop,Bar,Bookstore,Restaurant,Bakery,Japanese Restaurant,Noodle House,Comfort Food Restaurant,College Gym
2,Trinity,M6J,West Toronto,43.647927,-79.419750,True,1,Bar,Men's Store,Restaurant,Coffee Shop,Asian Restaurant,Café,Pizza Place,Cocktail Bar,Vietnamese Restaurant,Record Shop
7,Harbourfront East,M5J,Downtown Toronto,43.640816,-79.381752,True,2,Coffee Shop,Hotel,Aquarium,Café,Pizza Place,Brewery,Italian Restaurant,Restaurant,Scenic Lookout,Bakery
10,Ryerson,M5B,Downtown Toronto,43.657162,-79.378937,True,2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Thai Restaurant,Plaza,Diner,Tea Room,Theater
13,Stn A PO Boxes 25 The Esplanade,M5W,Downtown Toronto,43.646435,-79.374846,True,2,Coffee Shop,Restaurant,Café,Seafood Restaurant,Cocktail Bar,Pub,Hotel,Italian Restaurant,Bakery,Cosmetics Shop


 visualize the resulting clusters

In [27]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

Cluster 1

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,South Hill,Convenience Store,Coffee Shop,Pub,Light Rail Station,Supermarket,Fried Chicken Joint,Bagel Shop,Sports Bar,American Restaurant,Pizza Place
47,Toronto Islands,Coffee Shop,Hotel,Aquarium,Café,Pizza Place,Brewery,Italian Restaurant,Restaurant,Scenic Lookout,Bakery
102,Railway Lands,Airport Terminal,Airport Service,Airport Lounge,Boutique,Airport Gate,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport,Harbor / Marina
142,Chinatown,Café,Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Coffee Shop,Bakery,Chinese Restaurant,Mexican Restaurant,Comfort Food Restaurant,Gaming Cafe
168,High Park,Mexican Restaurant,Café,Bar,Park,Arts & Crafts Store,Fast Food Restaurant,Bookstore,Sandwich Place,Flea Market,Speakeasy
178,The Junction South,Mexican Restaurant,Café,Bar,Park,Arts & Crafts Store,Fast Food Restaurant,Bookstore,Sandwich Place,Flea Market,Speakeasy


Cluster 2

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Trinity,Bar,Men's Store,Restaurant,Coffee Shop,Asian Restaurant,Café,Pizza Place,Cocktail Bar,Vietnamese Restaurant,Record Shop
14,First Canadian Place,Café,Coffee Shop,Hotel,Restaurant,American Restaurant,Asian Restaurant,Gastropub,Gym,Seafood Restaurant,Deli / Bodega
64,Bathurst Quay,Airport Terminal,Airport Service,Airport Lounge,Boutique,Airport Gate,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport,Harbor / Marina
74,South Niagara,Airport Terminal,Airport Service,Airport Lounge,Boutique,Airport Gate,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport,Harbor / Marina
80,Forest Hill West,Trail,Jewelry Store,Park,Sushi Restaurant,Women's Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
95,Parkdale,Breakfast Spot,Gift Shop,Dog Run,Dessert Shop,Restaurant,Eastern European Restaurant,Bookstore,Italian Restaurant,Bank,Bar
146,St. James Town,Coffee Shop,Restaurant,Café,Hotel,Clothing Store,Cocktail Bar,Cosmetics Shop,Italian Restaurant,Gastropub,Seafood Restaurant


Cluster 3

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Harbord,Café,Coffee Shop,Bar,Bookstore,Restaurant,Bakery,Japanese Restaurant,Noodle House,Comfort Food Restaurant,College Gym
7,Harbourfront East,Coffee Shop,Hotel,Aquarium,Café,Pizza Place,Brewery,Italian Restaurant,Restaurant,Scenic Lookout,Bakery
10,Ryerson,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Thai Restaurant,Plaza,Diner,Tea Room,Theater
13,Stn A PO Boxes 25 The Esplanade,Coffee Shop,Restaurant,Café,Seafood Restaurant,Cocktail Bar,Pub,Hotel,Italian Restaurant,Bakery,Cosmetics Shop
17,Davisville,Sandwich Place,Pizza Place,Dessert Shop,Coffee Shop,Seafood Restaurant,Thai Restaurant,Italian Restaurant,Café,Sushi Restaurant,Restaurant
19,The Annex,Coffee Shop,Café,Sandwich Place,Pizza Place,Pharmacy,Indian Restaurant,Cosmetics Shop,Pub,Burger Joint,Metro Station
22,CN Tower,Airport Terminal,Airport Service,Airport Lounge,Boutique,Airport Gate,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport,Harbor / Marina
23,Garden District,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Thai Restaurant,Plaza,Diner,Tea Room,Theater
25,Commerce Court,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Seafood Restaurant,Steakhouse,Gastropub,Deli / Bodega
30,Swansea,Coffee Shop,Pizza Place,Café,Italian Restaurant,Sushi Restaurant,Electronics Store,Bookstore,Food & Drink Shop,Food,Bar


Cluster 4

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Rathnelly,Convenience Store,Coffee Shop,Pub,Light Rail Station,Supermarket,Fried Chicken Joint,Bagel Shop,Sports Bar,American Restaurant,Pizza Place
48,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Chinese Restaurant,Bar,Ice Cream Shop,Burger Joint,Sandwich Place,Thai Restaurant,Bubble Tea Shop
85,University of Toronto,Café,Coffee Shop,Bar,Bookstore,Restaurant,Bakery,Japanese Restaurant,Noodle House,Comfort Food Restaurant,College Gym
134,North Midtown,Coffee Shop,Café,Sandwich Place,Pizza Place,Pharmacy,Indian Restaurant,Cosmetics Shop,Pub,Burger Joint,Metro Station
166,Union Station,Coffee Shop,Hotel,Aquarium,Café,Pizza Place,Brewery,Italian Restaurant,Restaurant,Scenic Lookout,Bakery


Cluster 5

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
136,The Beaches West,Park,Pet Store,Sushi Restaurant,Ice Cream Shop,Italian Restaurant,Fish & Chips Shop,Fast Food Restaurant,Liquor Store,Movie Theater,Pizza Place


end